<a href="https://colab.research.google.com/github/sushmasridharindira/Semantic-Drift-Detection-in-Social-Media/blob/main/Semantic_Drift_Detection_in_Social_Media_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 4.2 MB/s eta 0:00:00


In [ ]:
#!/usr/bin/env python3
"""
Scrape Reddit posts for multiple keywords using PRAW.
Saves a combined CSV with columns: id, timestamp, text
"""

import praw
import pandas as pd
from datetime import datetime

# 🔑 Fill in your Reddit app credentials (script type)
CLIENT_ID = "kFNK7dZXGykyxkMn3YdI4g"
CLIENT_SECRET = "VmIj9F9n1qI_LYxIj0yrHysA3_VbMw"
USER_AGENT = "semantic-drift-research by u/Bubbly-Active-6675"

# Initialize Reddit API client
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT,
)

def scrape_keyword(keyword, subreddit="all", limit=500):
    """
    Scrape posts for a single keyword.

    Returns:
        pd.DataFrame with id, timestamp, text
    """
    posts = []
    for submission in reddit.subreddit(subreddit).search(keyword, sort="new", limit=limit):
        posts.append({
            "id": submission.id,
            "timestamp": datetime.utcfromtimestamp(submission.created_utc),
            "text": submission.title + " " + (submission.selftext or "")
        })
    return pd.DataFrame(posts)


if __name__ == "__main__":
    keywords = ["woke", "cloud", "nurse"]
    all_dfs = []

    for kw in keywords:
        print(f"Scraping posts for keyword: {kw}")
        df = scrape_keyword(kw, subreddit="all", limit=500)
        all_dfs.append(df)

    # Combine all keywords into one CSV
    combined_df = pd.concat(all_dfs, ignore_index=True).drop_duplicates(subset=["id"])
    combined_df.to_csv("reddit_combined.csv", index=False)
    print(f"✅ Saved {len(combined_df)} posts → reddit_combined.csv")


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Scraping posts for keyword: woke


/tmp/ipython-input-3974428366.py:34: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  "timestamp": datetime.utcfromtimestamp(submission.created_utc),
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Scraping posts for keyword: cloud


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Scraping posts for keyword: nurse


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



✅ Saved 736 posts → reddit_combined.csv


In [ ]:
#!/usr/bin/env python3
"""
Optimizations for Reddit:
- Clean markdown quotes, code blocks, and links
- Increased tokenizer max_length=512
- Works for posts and comments of variable lengths
"""

from __future__ import annotations
import argparse
import os
import json
from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Optional
import re
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import torch
from transformers import AutoTokenizer, AutoModel

# -----------------------------
# Utility & Config
# -----------------------------
@dataclass
class Config:
    data_path: str = "reddit_combined.csv"
    out_dir: str = "outputs_reddit"
    targets: List[str] = field(default_factory=lambda: ["woke", "cloud", "nurse"])
    model_name: str = "bert-base-uncased"
    max_docs: Optional[int] = None
    time_split: Optional[str] = None
    window_days: Optional[int] = None
    stride_days: Optional[int] = None
    min_occurrences_per_slice: int = 30
    clustering: str = "kmeans"
    kmeans_k: Optional[int] = None
    dbscan_eps: float = 0.8
    dbscan_min_samples: int = 10
    random_state: int = 42
    device: str = "cuda" if torch.cuda.is_available() else "cpu"


# -----------------------------
# Preprocessing for Reddit
# -----------------------------
def clean_reddit_text(text: str) -> str:
    """Remove markdown quotes, code blocks, and links from Reddit posts/comments."""
    text = re.sub(r"^>.*$", "", text, flags=re.MULTILINE)
    text = re.sub(r"```.*?```", "", text, flags=re.DOTALL)
    text = re.sub(r"http\S+|www.\S+", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


# -----------------------------
# Data Loading & Slicing
# -----------------------------
def load_corpus(path: str, max_docs: Optional[int] = None) -> pd.DataFrame:
    ext = os.path.splitext(path)[1].lower()
    if ext == ".csv":
        df = pd.read_csv(path)
    elif ext in {".jsonl", ".json"}:
        df = pd.read_json(path, lines=True)
    else:
        raise ValueError("Unsupported file extension. Use .csv or .jsonl")

    if "timestamp" not in df.columns or "text" not in df.columns:
        raise ValueError("Data must contain 'timestamp' and 'text' columns.")

    df = df.dropna(subset=["text", "timestamp"]).copy()
    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
    df = df.dropna(subset=["timestamp"])
    df = df.sort_values("timestamp")

    if max_docs is not None:
        df = df.head(max_docs)

    # Clean text for Reddit
    df["text"] = df["text"].astype(str).apply(clean_reddit_text)
    return df


def build_time_slices(
    df: pd.DataFrame,
    time_split: Optional[str] = None,
    window_days: Optional[int] = None,
    stride_days: Optional[int] = None,
) -> List[Tuple[str, pd.DatetimeIndex]]:

    if time_split:
        split = pd.to_datetime(time_split)
        mask1 = df["timestamp"] < split
        mask2 = df["timestamp"] >= split
        return [
            (f"pre_{split.date()}", df.index[mask1]),
            (f"post_{split.date()}", df.index[mask2]),
        ]

    elif window_days and stride_days:
        start = df["timestamp"].min().normalize()
        end = df["timestamp"].max().normalize()

        slices = []
        cur = start
        delta = pd.Timedelta(days=window_days)
        stride = pd.Timedelta(days=stride_days)

        while cur <= end:
            win_start = cur
            win_end = cur + delta
            mask = (df["timestamp"] >= win_start) & (df["timestamp"] < win_end)
            slices.append((f"{win_start.date()}_{win_end.date()}", df.index[mask]))
            cur += stride

        return slices

    else:
        raise ValueError("Provide either --time_split OR (--window_days and --stride_days).")


# -----------------------------
# Target Matching & Embeddings
# -----------------------------
def find_token_spans(text: str, target: str) -> List[Tuple[int, int]]:
    spans = []
    t_low = target.lower()
    low = text.lower()
    start = 0

    while True:
        i = low.find(t_low, start)
        if i == -1:
            break

        left_ok = i == 0 or not low[i - 1].isalnum()
        right_ok = i + len(t_low) == len(low) or not low[i + len(t_low)].isalnum()

        if left_ok and right_ok:
            spans.append((i, i + len(t_low)))

        start = i + len(t_low)

    return spans


def extract_token_embeddings(
    texts: List[str],
    target: str,
    tokenizer: AutoTokenizer,
    model: AutoModel,
    device: str = "cpu",
) -> np.ndarray:

    model.eval()
    all_embeds = []

    with torch.no_grad():
        for text in texts:
            spans = find_token_spans(text, target)
            if not spans:
                continue

            enc = tokenizer(
                text,
                return_tensors="pt",
                return_offsets_mapping=True,
                truncation=True,
                padding=False,
                max_length=512,
            )
            enc = {k: v.to(device) if hasattr(v, "to") else v for k, v in enc.items()}
            outputs = model(input_ids=enc["input_ids"], attention_mask=enc["attention_mask"])

            hidden = outputs.last_hidden_state.squeeze(0)
            offsets = enc["offset_mapping"].squeeze(0).cpu().numpy()

            for (s, e) in spans:
                token_idxs = [
                    ti for ti, (a, b) in enumerate(offsets)
                    if not (a == b) and not (b <= s or a >= e)
                ]
                if not token_idxs:
                    continue

                vec = hidden[token_idxs].mean(dim=0)
                all_embeds.append(vec.cpu().numpy())

    if not all_embeds:
        return np.empty((0, model.config.hidden_size))

    return np.vstack(all_embeds)


# -----------------------------
# Clustering & Metrics
# -----------------------------
def auto_kmeans_k(X: np.ndarray, k_min: int = 2, k_max: int = 5, random_state: int = 42) -> int:
    best_k, best_score = k_min, float("inf")

    for k in range(k_min, k_max + 1):
        km = KMeans(n_clusters=k, n_init=10, random_state=random_state)
        labels = km.fit_predict(X)

        score = 0.0
        for c in range(k):
            cluster_pts = X[labels == c]
            if len(cluster_pts) <= 1:
                continue

            dists = pairwise_distances(cluster_pts, [km.cluster_centers_[c]])
            score += dists.mean()

        score /= k

        if score < best_score:
            best_k, best_score = k, score

    return best_k


def cluster_embeddings(X: np.ndarray, cfg: Config) -> Tuple[np.ndarray, Dict]:
    if cfg.clustering == "kmeans":
        k = cfg.kmeans_k or auto_kmeans_k(X, 2, 5, cfg.random_state)
        km = KMeans(n_clusters=k, n_init=10, random_state=cfg.random_state)
        labels = km.fit_predict(X)
        info = {"centers": km.cluster_centers_.tolist(), "k": k}

    elif cfg.clustering == "dbscan":
        db = DBSCAN(eps=cfg.dbscan_eps, min_samples=cfg.dbscan_min_samples)
        labels = db.fit_predict(X)
        centers = [X[labels == c].mean(axis=0) for c in set(labels) if c != -1]
        info = {"centers": [c.tolist() for c in centers], "k": len(centers)}

    else:
        raise ValueError("Unknown clustering method")

    return labels, info


def centroid_distance(c1: np.ndarray, c2: np.ndarray) -> float:
    return float(np.linalg.norm(c1 - c2))


def js_divergence(p: np.ndarray, q: np.ndarray, eps: float = 1e-9) -> float:
    p = p.astype(float)
    q = q.astype(float)
    p = p / (p.sum() + eps)
    q = q / (q.sum() + eps)
    m = 0.5 * (p + q)

    def kl(a, b):
        a = np.clip(a, eps, 1)
        b = np.clip(b, eps, 1)
        return float((a * np.log(a / b)).sum())

    return 0.5 * kl(p, m) + 0.5 * kl(q, m)


def entropy(p: np.ndarray, eps: float = 1e-9) -> float:
    p = p.astype(float)
    p = p / (p.sum() + eps)
    p = np.clip(p, eps, 1)
    return float(-(p * np.log(p)).sum())


# -----------------------------
# Visualization & Timeline
# -----------------------------
def save_pca_scatter(X: np.ndarray, labels: np.ndarray, out_path: str, title: str = "PCA"):
    if X.shape[0] == 0:
        return

    scaler = StandardScaler()
    Xs = scaler.fit_transform(X)

    pca = PCA(n_components=2, random_state=42)
    X2 = pca.fit_transform(Xs)

    plt.figure(figsize=(6, 5))
    plt.scatter(X2[:, 0], X2[:, 1], s=6, alpha=0.7, c=labels)
    plt.title(title)
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.tight_layout()

    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    plt.savefig(out_path, dpi=180)
    plt.close()


def save_timeline(values: List[Tuple[pd.Timestamp, float]], out_path: str, title: str):
    if not values:
        return

    dates = [d for d, _ in values]
    vals = np.array([v for _, v in values], dtype=float)

    if len(vals) >= 5:
        mu, sigma = vals.mean(), vals.std() + 1e-9
        z = (vals - mu) / sigma
    else:
        z = vals

    plt.figure(figsize=(7, 3))
    plt.plot(dates, z, marker="o")
    plt.title(title + " (z-scored)")
    plt.xlabel("Time slice end")
    plt.ylabel("Drift signal (z)")
    plt.tight_layout()

    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    plt.savefig(out_path, dpi=180)
    plt.close()


# -----------------------------
# Orchestration
# -----------------------------
def run_pipeline(cfg: Config):
    os.makedirs(cfg.out_dir, exist_ok=True)

    fig_dir = os.path.join(cfg.out_dir, "figures")
    emb_dir = os.path.join(cfg.out_dir, "embeddings")
    met_dir = os.path.join(cfg.out_dir, "metrics")
    for d in [fig_dir, emb_dir, met_dir]:
        os.makedirs(d, exist_ok=True)

    print("Loading corpus…")
    df = load_corpus(cfg.data_path, cfg.max_docs)

    slices = build_time_slices(df, cfg.time_split, cfg.window_days, cfg.stride_days)

    print(f"Loading model: {cfg.model_name}")
    tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)
    model = AutoModel.from_pretrained(cfg.model_name).to(cfg.device)

    for target in cfg.targets:
        print(f"\n=== Target: {target} ===")
        slice_results = []

        for slice_name, idx in slices:
            texts = df.loc[idx, "text"].tolist()
            embeds = extract_token_embeddings(texts, target, tokenizer, model, cfg.device)

            print(f"Slice {slice_name}: {embeds.shape[0]} occurrences")

            if embeds.shape[0] < cfg.min_occurrences_per_slice:
                print(" -> Skipping (too few occurrences).")
                slice_results.append((slice_name, embeds, None, {"k": 0, "centers": []}))
                continue

            labels, info = cluster_embeddings(embeds, cfg)

            np.save(os.path.join(emb_dir, f"{target}_{slice_name}.npy"), embeds)
            save_pca_scatter(
                embeds,
                labels,
                os.path.join(fig_dir, f"{target}_pca_{slice_name}.png"),
                title=f"{target} – {slice_name} ({cfg.clustering})",
            )

            slice_results.append((slice_name, embeds, labels, info))

        # ----- Compute Metrics -----
        metrics: Dict[str, Dict] = {}

        # centroid distance
        consecutive_centroids = []
        for slice_name, embeds, labels, info in slice_results:
            c = np.array(embeds).mean(axis=0) if embeds.size else None
            consecutive_centroids.append((slice_name, c))

        centroid_dists = []
        for i in range(1, len(consecutive_centroids)):
            prev_name, c1 = consecutive_centroids[i - 1]
            cur_name, c2 = consecutive_centroids[i]
            dist = centroid_distance(c1, c2) if c1 is not None and c2 is not None else None
            centroid_dists.append(((prev_name, cur_name), dist))

        # JS divergence + entropy delta
        js_list = []
        ent_list = []

        for i in range(1, len(slice_results)):
            (name1, _E1, lab1, info1) = slice_results[i - 1]
            (name2, _E2, lab2, info2) = slice_results[i]

            js_val = None
            ent_delta = None

            if (
                lab1 is not None and lab2 is not None
                and info1.get("k", 0) > 0
                and info2.get("k", 0) > 0
            ):
                k = max(info1["k"], info2["k"])

                def hist(lab, k):
                    valid = lab[lab >= 0] if isinstance(lab, np.ndarray) else np.array(
                        [x for x in lab if x >= 0]
                    )
                    h = np.zeros(k)
                    if valid.size:
                        counts = np.bincount(valid, minlength=k)
                        h[: len(counts)] = counts
                    return h

                h1, h2 = hist(lab1, k), hist(lab2, k)

                js_val = js_divergence(h1, h2)
                ent_delta = abs(entropy(h2) - entropy(h1))

            js_list.append(((name1, name2), js_val))
            ent_list.append(((name1, name2), ent_delta))

        metrics["centroid_distance_consecutive"] = [
            {"from": a, "to": b, "value": (None if v is None else float(v))}
            for (a, b), v in centroid_dists
        ]

        metrics["js_divergence_consecutive"] = [
            {"from": a, "to": b, "value": (None if v is None else float(v))}
            for (a, b), v in js_list
        ]

        metrics["entropy_delta_consecutive"] = [
            {"from": a, "to": b, "value": (None if v is None else float(v))}
            for (a, b), v in ent_list
        ]

        # ----- Timeline drift (if sliding windows) -----
        if cfg.window_days and cfg.stride_days:
            timeline_vals: List[Tuple[pd.Timestamp, float]] = []

            for ((a, b), v), (slice_name, idx) in zip(centroid_dists, slices[1:]):
                end_date = df.loc[idx, "timestamp"].max() if len(idx) else pd.NaT
                if pd.isna(end_date) or v is None:
                    continue

                timeline_vals.append((end_date, float(v)))

            save_timeline(
                timeline_vals,
                os.path.join(fig_dir, f"{target}_timeline.png"),
                title=f"Early drift for '{target}'",
            )

            metrics["early_drift_points"] = [
                {"date": str(d.date()), "signal": float(v)}
                for d, v in timeline_vals
            ]

        with open(os.path.join(met_dir, f"{target}_metrics.json"), "w") as f:
            json.dump(metrics, f, indent=2)

        print(f"Saved metrics for {target} -> {os.path.join(met_dir, f'{target}_metrics.json')}")


# -----------------------------
# CLI
# -----------------------------
def parse_args() -> Config:
    p = argparse.ArgumentParser(description="Semantic Drift Detection Pipeline for Reddit")
    p.add_argument("--data_path", required=True)
    p.add_argument("--out_dir", required=True)
    p.add_argument("--targets", nargs="+", required=True)
    p.add_argument("--model", default="bert-base-uncased")
    p.add_argument("--max_docs", type=int, default=None)
    p.add_argument("--time_split", type=str, default=None)
    p.add_argument("--window_days", type=int, default=None)
    p.add_argument("--stride_days", type=int, default=None)
    p.add_argument("--min_occurrences_per_slice", type=int, default=30)
    p.add_argument("--clustering", choices=["kmeans", "dbscan"], default="kmeans")
    p.add_argument("--kmeans_k", type=int, default=None)
    p.add_argument("--dbscan_eps", type=float, default=0.8)
    p.add_argument("--dbscan_min_samples", type=int, default=10)
    p.add_argument("--seed", type=int, default=42)

    args = p.parse_args()

    return Config(
        data_path=args.data_path,
        out_dir=args.out_dir,
        targets=args.targets,
        model_name=args.model,
        max_docs=args.max_docs,
        time_split=args.time_split,
        window_days=args.window_days,
        stride_days=args.stride_days,
        min_occurrences_per_slice=args.min_occurrences_per_slice,
        clustering=args.clustering,
        kmeans_k=args.kmeans_k,
        dbscan_eps=args.dbscan_eps,
        dbscan_min_samples=args.dbscan_min_samples,
        random_state=args.seed,
    )


# -----------------------------
# Main
# -----------------------------
if __name__ == "__main__":
    config = Config(
        data_path="reddit_combined.csv",
        out_dir="outputs_reddit",
        targets=["woke", "cloud", "nurse"],
        window_days=1,
        stride_days=1,
        min_occurrences_per_slice=1,
    )
    run_pipeline(config)

Loading corpus…
Loading model: bert-base-uncased


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]


=== Target: woke ===
Slice 2025-11-26_2025-11-27: 228 occurrences
Saved metrics for woke -> outputs_reddit/metrics/woke_metrics.json

=== Target: cloud ===
Slice 2025-11-26_2025-11-27: 290 occurrences
Saved metrics for cloud -> outputs_reddit/metrics/cloud_metrics.json

=== Target: nurse ===
Slice 2025-11-26_2025-11-27: 149 occurrences
Saved metrics for nurse -> outputs_reddit/metrics/nurse_metrics.json
